# Process Raw NSC files to Warehouse

This notebook executes the following actions:
1. Uploads the NSC files to Drive. This includes the contents of the zipped folder.
2. Upload the CSV file to the warehouse
3. Process the uploaded data to `public.college_enrollments` and `public.college_degrees`

## Make sure the CSV is raw
Do **NOT** open the CSV and hit save prior to running this notebook.

In [1]:
import os
import shutil

from zipfile import ZipFile

from spswarehouse.warehouse import Warehouse
from spswarehouse.googledrive import GoogleDrive
from spswarehouse.table_utils import *

Warehouse.execute("USE ROLE dataops")

To access your Google Drive file, share the file with jupyter-sheets@sps-warehouse.iam.gserviceaccount.com


(<sqlalchemy.engine.base.Connection at 0x18f716dea30>,
 <sqlalchemy.engine.cursor.LegacyCursorResult at 0x18f76ad0d90>)

# Setup

It's easiest to move the two files from NSC into the same directory as this notebook.

In [2]:
# Below format assumes you moved the two NSC files to this directory
excel_file_path = '10055266hsst_10055266-186540-DETAIL-EFFDT-20240821-RUNDT-20240905.csv'
zip_file_path = '10055266hsst_10055266_EFFDT_20240821_RUNDT_20240905143257301.zip'

# The "as of" date for the NSC file. The NSC file name contains two dates - this is the first date.
# (The second date is the day the file was generated, but it's still a snapshot as of the first date)
nsc_date = '2024-08-21'

In [3]:
# Dates of all available NSC data in the warehouse, with underscores
# Add the date you are working on right now
list_of_all_nsc_dates = [
    '2024_08_21',
    '2024_04_03',
    '2023_08_17',
    '2023_04_27',
    '2022_12_05',
    '2022_08_30',
    '2022_04_21',
    '2021_04_16',
    '2020_04_22',
    '2020_09_18',
    '2019_04_15',
    '2019_11_25',
    '2019_08_17',
    '2018_11_28',
]

## Permanent Setup
These variables should not change with time

In [4]:
# hyphens in warehouse table names are big PITA
date_underscored = nsc_date.replace("-", "_")

schema = 'national_student_clearinghouse'
raw_table = 'raw_data_' + date_underscored
clean_table = 'clean_data_' + date_underscored

clean_table_sql_file = './Warehouse_SQL/raw_to_clean_nsc_python.sql'
enrollment_sql_file = './Warehouse_SQL/public_enrollments_python.sql'
degree_sql_file = './Warehouse_SQL/public_degrees_python.sql'

# NSC files permanent home is https://drive.google.com/drive/folders/1Y3bSyRHwceFmsNYbo9vd59UXEtZHpj99
alumni_folder_id = '1Y3bSyRHwceFmsNYbo9vd59UXEtZHpj99'

# Number of days between enrollment records to indicate a dropout
# Must be long enough to account for summer
enrollment_gap = 131

In [5]:
def UploadFileToDrive(filename, parent_folder_id):
    upload_file = GoogleDrive.CreateFile({
        'title': filename,
        'parents': [{"kind": "drive#fileLink", "id": parent_folder_id}],
    })
    upload_file.SetContentFile(filename)
    upload_file.Upload()
    return upload_file

# Upload files to Drive

## Extract all files from the zip folder

In [6]:
temp_dir = 'data'

nsc_zip = ZipFile('./' + zip_file_path)

# Note: you can skip os.makedir because extractall creates the given path if it doesn't exist
nsc_zip.extractall(temp_dir)

nsc_zip.close()

NameError: name 'zip_file_path' is not defined

## Retrieve list of CEEB codes

In [7]:
ceeb_sql = """
SELECT DISTINCT
    site_short_name
    , ceeb_code
FROM public.sites_historical
"""

sites_df = Warehouse.read_sql(ceeb_sql)

## Create Drive folder for this upload, upload raw data files there

In [8]:
folder_name = nsc_date + '_nsc_files'

newFolder = GoogleDrive.CreateFile({
    'title': folder_name,
    "parents": [{"kind": "drive#fileLink", "id": alumni_folder_id}],
    "mimeType": "application/vnd.google-apps.folder"
})

newFolder.Upload()

In [9]:
csv_drive_file = UploadFileToDrive(excel_file_path, newFolder['id'])
UploadFileToDrive(zip_file_path, newFolder['id'])

NameError: name 'zip_file_path' is not defined

## Rename Files

In [10]:
os.chdir(temp_dir)

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'data'

In [ ]:
zip_file_list = os.listdir()
for file_name in zip_file_list:
    if 'ACADEMICS' in file_name:
        os.remove(file_name)
        continue

    if file_name[22:24] == 'HS':
        ceeb_code = file_name[24:30]
        site_name = sites_df[sites_df['ceeb_code']==ceeb_code]['site_short_name'].iloc[0]
        os.rename(file_name, site_name + file_name[37:])
    else:
        os.rename(file_name, 'Network' + file_name[37:])

## Upload renamed files to drive

In [ ]:
# upload re-named files
upload_file_list = os.listdir()
for file_name in upload_file_list:
    file_upload = UploadFileToDrive(file_name, newFolder['id'])
        

## Cleanup

In [11]:
os.chdir("../")
# Verify that you are where you think you are!
print(os.getcwd())

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data'

In [ ]:
# This might not work - I can't seem to close the connection to the last file uploaded
# If it doesn't, manually delete the `data` folder and the two NSC files from the folder
shutil.rmtree('./' + temp_dir)

# Upload to Warehouse

In [12]:
drive_file_id = csv_drive_file['id']

Warehouse.execute("USE ROLE dataops")

create_sql = create_table_stmt(raw_table, schema, google_drive_id=drive_file_id, force_string=True, encoding='latin-1')
Warehouse.execute(create_sql)

(<sqlalchemy.engine.base.Connection at 0x18f716dea30>,
 <sqlalchemy.engine.cursor.LegacyCursorResult at 0x18f76d5eee0>)

In [13]:
table_reflect = Warehouse.reflect(raw_table, schema)

upload_to_warehouse(table_reflect, google_drive_id=drive_file_id, force_string=True, encoding= 'latin-1', batch_size=2000)

28933 rows to insert
Inserted 2000 rows to national_student_clearinghouse.raw_data_2024_04_03
Inserted 2000 rows to national_student_clearinghouse.raw_data_2024_04_03
Inserted 2000 rows to national_student_clearinghouse.raw_data_2024_04_03
Inserted 2000 rows to national_student_clearinghouse.raw_data_2024_04_03
Inserted 2000 rows to national_student_clearinghouse.raw_data_2024_04_03
Inserted 2000 rows to national_student_clearinghouse.raw_data_2024_04_03
Inserted 2000 rows to national_student_clearinghouse.raw_data_2024_04_03
Inserted 2000 rows to national_student_clearinghouse.raw_data_2024_04_03
Inserted 2000 rows to national_student_clearinghouse.raw_data_2024_04_03
Inserted 2000 rows to national_student_clearinghouse.raw_data_2024_04_03
Inserted 2000 rows to national_student_clearinghouse.raw_data_2024_04_03
Inserted 2000 rows to national_student_clearinghouse.raw_data_2024_04_03
Inserted 2000 rows to national_student_clearinghouse.raw_data_2024_04_03
Inserted 2000 rows to national

# Create Cleaned Data Table

In [14]:
clean_table_sql = open(clean_table_sql_file).read()

formatted_clean_sql = clean_table_sql.format(
    clean_table=clean_table,
    enrollment_gap=enrollment_gap,
    raw_table=raw_table,
    schema=schema,
    update_date=nsc_date,
)
Warehouse.execute(formatted_clean_sql)

(<sqlalchemy.engine.base.Connection at 0x18f716dea30>,
 <sqlalchemy.engine.cursor.LegacyCursorResult at 0x18f76f014c0>)

# Refresh public tables

In [15]:
# public.college_enrollments

# Create the series of statements for the UNION
enrollment_union_sql = ""

for upload_date in list_of_all_nsc_dates:
    sql = f"""
    SELECT *
    FROM {schema}.clean_data_{upload_date}
    UNION"""
    
    enrollment_union_sql += sql
    
enrollment_sql = open(enrollment_sql_file).read()
formatted_enrollment_sql = enrollment_sql.format(
    union_sql=enrollment_union_sql
)

Warehouse.execute(formatted_enrollment_sql)

(<sqlalchemy.engine.base.Connection at 0x18f716dea30>,
 <sqlalchemy.engine.cursor.LegacyCursorResult at 0x18f76f01040>)

In [16]:
# public.college_degrees

# Create the series of statements for the UNION
degree_union_sql = ""

for upload_date in list_of_all_nsc_dates:
    upload_date_hyphens = upload_date.replace('_', '-')
    sql = f"""
    SELECT *, '{upload_date_hyphens}' AS date_last_updated
    FROM {schema}.raw_data_{upload_date}
    UNION"""
    
    degree_union_sql += sql
    
degree_sql = open(degree_sql_file).read()
formatted_degree_sql = degree_sql.format(
    union_sql=degree_union_sql
)

Warehouse.execute(formatted_degree_sql)

(<sqlalchemy.engine.base.Connection at 0x18f716dea30>,
 <sqlalchemy.engine.cursor.LegacyCursorResult at 0x18f76f01610>)